# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [1]:
from string import punctuation
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter as C
from scipy.sparse import lil_matrix, csc_matrix
import numpy as np

Данные

In [2]:
with open("lenta.txt", encoding='utf8') as f:
    corpus = f.read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                       in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in \
                       normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
corpus_sent = []
for sent in tqdm(sent_tokenize(corpus[:5000000])):
    corpus_sent.append(['<start>'] * 2 + normalize(sent) + ['<end>'])

100%|██████████████████████████████████████████████████████████████████████████| 32250/32250 [00:04<00:00, 6735.67it/s]


In [5]:
print(*corpus_sent[0])

<start> <start> бои у сопоцкина и друскеник закончились отступлением германцев <end>


In [6]:
train, test = train_test_split(corpus_sent, test_size=20)

Подготовка к генерации

In [7]:
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(" ".join(tokens[i:i+n]))
    return ngrams

In [8]:
unigrams = C()
bigrams = C()
trigrams = C()

for sent in tqdm(train):
    unigrams.update(sent)
    bigrams.update(ngrammer(sent, n=2))
    trigrams.update(ngrammer(sent, n=3))

100%|█████████████████████████████████████████████████████████████████████████| 32230/32230 [00:01<00:00, 27849.71it/s]


In [9]:
matrix_bi_uni = lil_matrix(
    (len(bigrams), len(unigrams))
)

id2word = list(unigrams)
word2id_ = {word:i for i, word in enumerate(id2word)}

id2bigram_ = list(bigrams)
bigram2id = {bigram:i for i, bigram in enumerate(id2bigram_)}

for trigram in tqdm(trigrams):
    bigram, word = trigram.rsplit(" ", 1)
    matrix_bi_uni[bigram2id[bigram], word2id_[word]] = \
        (trigrams[trigram] / bigrams[bigram])

matrix_bi_uni = csc_matrix(matrix_bi_uni)

100%|██████████████████████████████████████████████████████████████████████| 551592/551592 [00:02<00:00, 248689.00it/s]


In [10]:
matrix_bi_uni

<380157x71955 sparse matrix of type '<class 'numpy.float64'>'
	with 551592 stored elements in Compressed Sparse Column format>

In [11]:
def generate(matrix, id2word, bigram2id, n=200):
    text = ['<start>', '<start>']
    current_idx = bigram2id['<start> <start>']
    for i in range(n):
        # choose random idx to continue, append the word to text
        chosen_idx = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        chosen = id2word[chosen_idx]
        text.append(chosen)
        if chosen != '<end>':
            current_idx = bigram2id[text[-2] + ' ' + text[-1]]
        else:
            current_idx = bigram2id['<start> <start>']
            text.extend(['<start>', '<start>'])
    return ' '.join(text)

Пробная генерация

In [12]:
print(generate(matrix_bi_uni, id2word, bigram2id).replace('<end>', '\n').replace('<start> <start>', ''))

 еще две упаковки со взрывчаткой прорваться из горных районов чечни для встречи нового года когда он в интервью риа новости сообщили в центре таллина трое преступников в масках ворвались в салон ранили водителя забрали у пассажиров 60 тысяч рублей и в великую отечественную войну 
  с точки зрения эмоциональной привлекательности продуктов и те же годы реставрационные работы в наиболее тяжелом состоянии с нейсправиться сказал министр для беженцевв ингушетию будут доставлены в пригород бишкека 
  что касается представителей организаций борющихся за права больных спидом 
  как заявили интерфаксу в российском информационном пресс-центре объявил начальник центра общественных связей фсб россии по ростовской области добавляет рбк полностью отрицают версию о том что были предотвращены взрывы на железнодорожных станциях гудермес терек наурская червленая 
  однако примет ли милошевич это приглашение 
  я думаю что мы делаем абсолютно соответствует нормам законаи конституции подчеркнул он 
  такое

Основное наблюдение -- это то, что после конца предложения, согласно нашей процедуре, могут получаться предложения, которые вообще не связаны с предыдущим

Подготовка к оценке

In [13]:
def compute_joint_proba_markov(tokenized_text, word_counts, bigram_counts, trigram_counts):
    prob = 0
    for trigram in ngrammer(['<start>'] * 2 + tokenized_text + ['<end>'], n=3):
        word1, word2, word3 = trigram.split()
        bigram = word1 + ' ' + word2
        if bigram in bigram_counts and trigram in trigram_counts:
            prob += np.log(trigram_counts[trigram] / bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    return prob, len(tokenized_text)

In [14]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [15]:
print(*test[0])

<start> <start> это дебют мендеса в кино <end>


In [16]:
perplexity(*compute_joint_proba_markov(test[0], unigrams, bigrams, trigrams))

86654.40493966118

Оценка средней перплексии

In [17]:
mean_perpl = 0
for i in tqdm(range(len(test))):
    perpl = perplexity(*compute_joint_proba_markov(test[i], unigrams, bigrams, trigrams))
    mean_perpl += perpl
    if i < 10:
        print(f'Test {i}, perplexity {round(perpl, 3)}')
        print()
        print(*test[i])
        print()
        print()

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 740.82it/s]

Test 0, perplexity 86654.405

<start> <start> это дебют мендеса в кино <end>


Test 1, perplexity 18201.565

<start> <start> таким образом поскольку праздничный день 7 ноября объявленный президентом рф днем согласия и примирения совпадает с выходным понедельник 8 ноября становится нерабочим днем 5 ноября вступил в силу приговор головинского межмуниципального суда москвы в отношении музыканта ильи гофмана сообщает риа новости <end>


Test 2, perplexity 480.124

<start> <start> как сообщает риа новости приговор был вынесен с учетом личности подсудимого состояния его здоровья и того обстоятельства что гофман раскаялся первый заместитель начальника генштаба вооруженных сил рф генерал-полковник валерий манилов назвал в пятницу на брифинге в росинформцентре цифры потерь федеральных войск на северном кавказе с начала боевых действий в дагестане до настоящего времени <end>


Test 3, perplexity 27241.344

<start> <start> по данным гершвина больше остальных стран от проблемы-2000 могут пострадат

In [18]:
round(mean_perpl / len(test), 3)

22320.925

Могло бы быть и лучше, но неплохо. Есть доля случайности в выборке (при делении на "обучающую" и тестовую)

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [19]:
# pass